In [1]:
%pip install openai faiss-cpu numpy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import faiss
import numpy as np
from openai import OpenAI

In [3]:
# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key>"))
client = OpenAI()

In [4]:
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text, model="text-embedding-3-small"
    )
    embeddings = response.data[0].embedding
    return embeddings

In [5]:
embedding = get_vector_embeddings("information retrieval with a FAISS vector database")
print(embedding)

[-0.04561610519886017, 0.0031051726546138525, 0.03684069216251373, -0.046231456100940704, 0.025068793445825577, -0.004354264587163925, -0.012240098789334297, 0.010046245530247688, -0.013250074349343777, 0.04754241928458214, 0.010882318019866943, -0.08770064264535904, 0.02217932790517807, -0.011323763988912106, 0.00858144648373127, 0.0092436159029603, 0.05059241130948067, 0.0018192933639511466, -0.03726876154541969, 9.525999485049397e-05, 0.02278129942715168, -0.016400394961237907, -0.01538373064249754, -0.02291507087647915, -0.005411060061305761, -0.02509554661810398, 0.02427954040467739, 0.033656928688287735, 0.02690146304666996, -0.03852621465921402, -0.008420920930802822, -0.04920118674635887, -0.007671799976378679, -0.004645218141376972, -0.01609271951019764, 0.055702485144138336, -0.011905670166015625, -0.01724315620958805, -0.0028209080919623375, 0.01763109304010868, -0.011036154814064503, 0.032024916261434555, 0.021737881004810333, -0.05179635435342789, 0.053909946233034134, -0.

In [8]:
def chunk_text(text, window_size=5, stride=2):
    sentences = re.split('(?<=[.!?]) +', text.strip())
    chunks = []

    # Slide the window over the sentences
    for i in range(0, len(sentences) - window_size + 1, stride):
        # Add the current window to the chunks
        chunks.append(' '.join(sentences[i:i + window_size]))

    return chunks

In [9]:
# 출처 : https://m.dongascience.com/news.php?idx=59735
text = """
나폴레옹의 장점이라면 첫째는 통솔력이다. 엘바섬을 탈출하여 파리로 향할 때 루이 18세의 명을 받아 나폴레옹을 잡으러 간 병사들은 나폴레옹의 편에 서서 그의 재집권을 도왔다. 
그의 부하들은 목숨을 두려워하지 않고 그의 명령을 따를 정도였다. 세상을 마음대로 휘두르던 그였지만 전쟁터에서 그는 병사들과 똑같은 음식을 먹었다. 
그의 군대의 특징이라면 건강한 이들을 선두에 세우고 군장을 맨 상태에서 하루에 30km에 가까운 거리를 걸었다. 이렇게 빨리 걸었으니 후방에서 음식을 전해 주는 것은 거의 불가능했다.
나폴레옹은 군인들이 제대로 먹지 못하면, 특히 침략국이 식량을 팔거나 제공할 능력이 없거나 그런 의도조차 없다면 침략은 했지만 피침국민들의 인정을 받을 수 없다고 생각했다. 
그래서 군인들이 직접 음식을 가지고 가게 했다. 이를 위해 개발된 것이 통조림이었다.
통조림이 개발되기 전이었으므로 나폴레옹은 군인들이 음식을 가지고 다니면서 먹을 수 있도록 오랜 기간 보존 가능한 방법을 제안하는 사람에게 1만2000프랑의 상금을 걸었다. 
이 상의 수상자는 15년 후에나 탄생했고, ‘음식과학의 아버지(통조림의 아버지라고도 함)’라 불리는 아페르(Nicolas François Appert)가 그 주인공이다.
아페르는 1804년, 파리 근처에 식품병 공장을 짓고, 밀봉된 병에 여러가지 식품을 보존하는 사업을 시작했다. 통조림이라면 양철을 쉽게 떠올릴 수 있지만 
19세기 초의 프랑스 양철 제품은 품질이 좋지 않았으므로 아페르는 유리병을 이용했다.
그가 포장한 방법은 병 위쪽에 약간의 공간을 남겨둔 채로 코르크마개로 병을 밀봉하는 방식이었다. 그 후에 병을 끓는 물에 담가 두는 방법으로 요리를 했다. 
이 때는 미생물에 대한 개념이 없던 시기였으므로 아페르의 통찰력이 뛰어났음을 엿볼 수 있다. 
아페르는 작은 병 대신 큰 항아리를 이용하기도 했고, 밀봉 후에 항아리를 캔버스로 감싼 후 물에 넣고 끓이기도 했다. 
그의 방법은 1806년에 프랑스 해군들에게 음식을 공급하기 위해 처음 사용되었다.
"""

In [10]:
chunks = chunk_text(text, window_size=4, stride=1)
print(f"chunk length : {len(chunks)}")

chunk length : 11


In [11]:
chunks[0]

'나폴레옹의 장점이라면 첫째는 통솔력이다. 엘바섬을 탈출하여 파리로 향할 때 루이 18세의 명을 받아 나폴레옹을 잡으러 간 병사들은 나폴레옹의 편에 서서 그의 재집권을 도왔다. \n그의 부하들은 목숨을 두려워하지 않고 그의 명령을 따를 정도였다. 세상을 마음대로 휘두르던 그였지만 전쟁터에서 그는 병사들과 똑같은 음식을 먹었다.'

In [12]:
chunks[1]

'엘바섬을 탈출하여 파리로 향할 때 루이 18세의 명을 받아 나폴레옹을 잡으러 간 병사들은 나폴레옹의 편에 서서 그의 재집권을 도왔다. \n그의 부하들은 목숨을 두려워하지 않고 그의 명령을 따를 정도였다. 세상을 마음대로 휘두르던 그였지만 전쟁터에서 그는 병사들과 똑같은 음식을 먹었다. \n그의 군대의 특징이라면 건강한 이들을 선두에 세우고 군장을 맨 상태에서 하루에 30km에 가까운 거리를 걸었다.'

In [13]:
chunks[2]

'\n그의 부하들은 목숨을 두려워하지 않고 그의 명령을 따를 정도였다. 세상을 마음대로 휘두르던 그였지만 전쟁터에서 그는 병사들과 똑같은 음식을 먹었다. \n그의 군대의 특징이라면 건강한 이들을 선두에 세우고 군장을 맨 상태에서 하루에 30km에 가까운 거리를 걸었다. 이렇게 빨리 걸었으니 후방에서 음식을 전해 주는 것은 거의 불가능했다.\n나폴레옹은 군인들이 제대로 먹지 못하면, 특히 침략국이 식량을 팔거나 제공할 능력이 없거나 그런 의도조차 없다면 침략은 했지만 피침국민들의 인정을 받을 수 없다고 생각했다.'

In [14]:
# Get vector embeddings for the chunks
vectors = np.array([get_vector_embeddings(chunk) for chunk in chunks])
print(vectors.shape)

(11, 1536)


In [15]:
# Create a FAISS index
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

In [16]:
# Vector search function
def vector_search(query_text, k=3):
    query_vector = get_vector_embeddings(query_text)
    distances, indices = index.search(np.array([query_vector]), k)
    return [(chunks[i], float(dist)) for dist, i in zip(distances[0], indices[0])]

In [17]:
# Example search
search_word = "아페르"
search_results = vector_search(search_word)
print(f"Search results for {search_word} :")
for result in search_results:
    print(result)

Search results for 아페르 :
('\n이 상의 수상자는 15년 후에나 탄생했고, ‘음식과학의 아버지(통조림의 아버지라고도 함)’라 불리는 아페르(Nicolas François Appert)가 그 주인공이다.\n아페르는 1804년, 파리 근처에 식품병 공장을 짓고, 밀봉된 병에 여러가지 식품을 보존하는 사업을 시작했다. 통조림이라면 양철을 쉽게 떠올릴 수 있지만 \n19세기 초의 프랑스 양철 제품은 품질이 좋지 않았으므로 아페르는 유리병을 이용했다.\n그가 포장한 방법은 병 위쪽에 약간의 공간을 남겨둔 채로 코르크마개로 병을 밀봉하는 방식이었다. 그 후에 병을 끓는 물에 담가 두는 방법으로 요리를 했다. \n이 때는 미생물에 대한 개념이 없던 시기였으므로 아페르의 통찰력이 뛰어났음을 엿볼 수 있다.', 1.2439948320388794)
('이렇게 빨리 걸었으니 후방에서 음식을 전해 주는 것은 거의 불가능했다.\n나폴레옹은 군인들이 제대로 먹지 못하면, 특히 침략국이 식량을 팔거나 제공할 능력이 없거나 그런 의도조차 없다면 침략은 했지만 피침국민들의 인정을 받을 수 없다고 생각했다. \n그래서 군인들이 직접 음식을 가지고 가게 했다. 이를 위해 개발된 것이 통조림이었다.\n통조림이 개발되기 전이었으므로 나폴레옹은 군인들이 음식을 가지고 다니면서 먹을 수 있도록 오랜 기간 보존 가능한 방법을 제안하는 사람에게 1만2000프랑의 상금을 걸었다. \n이 상의 수상자는 15년 후에나 탄생했고, ‘음식과학의 아버지(통조림의 아버지라고도 함)’라 불리는 아페르(Nicolas François Appert)가 그 주인공이다.\n아페르는 1804년, 파리 근처에 식품병 공장을 짓고, 밀봉된 병에 여러가지 식품을 보존하는 사업을 시작했다.', 1.2710378170013428)
('통조림이라면 양철을 쉽게 떠올릴 수 있지만 \n19세기 초의 프랑스 양철 제품은 품질이 좋지 않았으므로 아페르는 유리병을 이용했다.\n그가 포장한 방법은 병 위쪽에 약간의 공간을 남겨둔 

In [18]:
# Function to perform a vector search and then ask a question to GPT
def search_and_chat(chat_prompt, k=3):
    search_results = vector_search(chat_prompt, k)
    print(f"Search results: {search_results}\n\n")

    prompt_with_context = f"Context:{search_results}\n Answer the question: {chat_prompt}"
    print(f"prompt with context: {prompt_with_context}")

    # Create a list of messages for the chat
    messages = [
      {"role": "system", "content": "Please answer the questions provided by the user. Use only the context provided to you, if you don't know the answer say \"I don't know\"."},
      {"role": "user", "content": prompt_with_context},
    ]

    # Get the model's response
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    print(f"Response: {response.choices[0].message.content}")

In [19]:
# Example search and chat
search_and_chat("누가 통조림을 만들었으며, 그 사람이 받은 상금은 얼마인가요?")

Search results: [('이를 위해 개발된 것이 통조림이었다.\n통조림이 개발되기 전이었으므로 나폴레옹은 군인들이 음식을 가지고 다니면서 먹을 수 있도록 오랜 기간 보존 가능한 방법을 제안하는 사람에게 1만2000프랑의 상금을 걸었다. \n이 상의 수상자는 15년 후에나 탄생했고, ‘음식과학의 아버지(통조림의 아버지라고도 함)’라 불리는 아페르(Nicolas François Appert)가 그 주인공이다.\n아페르는 1804년, 파리 근처에 식품병 공장을 짓고, 밀봉된 병에 여러가지 식품을 보존하는 사업을 시작했다. 통조림이라면 양철을 쉽게 떠올릴 수 있지만 \n19세기 초의 프랑스 양철 제품은 품질이 좋지 않았으므로 아페르는 유리병을 이용했다.\n그가 포장한 방법은 병 위쪽에 약간의 공간을 남겨둔 채로 코르크마개로 병을 밀봉하는 방식이었다. 그 후에 병을 끓는 물에 담가 두는 방법으로 요리를 했다.', 1.1932806968688965), ('\n그래서 군인들이 직접 음식을 가지고 가게 했다. 이를 위해 개발된 것이 통조림이었다.\n통조림이 개발되기 전이었으므로 나폴레옹은 군인들이 음식을 가지고 다니면서 먹을 수 있도록 오랜 기간 보존 가능한 방법을 제안하는 사람에게 1만2000프랑의 상금을 걸었다. \n이 상의 수상자는 15년 후에나 탄생했고, ‘음식과학의 아버지(통조림의 아버지라고도 함)’라 불리는 아페르(Nicolas François Appert)가 그 주인공이다.\n아페르는 1804년, 파리 근처에 식품병 공장을 짓고, 밀봉된 병에 여러가지 식품을 보존하는 사업을 시작했다. 통조림이라면 양철을 쉽게 떠올릴 수 있지만 \n19세기 초의 프랑스 양철 제품은 품질이 좋지 않았으므로 아페르는 유리병을 이용했다.\n그가 포장한 방법은 병 위쪽에 약간의 공간을 남겨둔 채로 코르크마개로 병을 밀봉하는 방식이었다.', 1.2340691089630127), ('\n그의 군대의 특징이라면 건강한 이들을 선두에 세우고 군장을 맨 상태에서 하루에 30km에 가까운